# Cómo usar PySAT y no morir en el intento

PySAT es una herramienta en Python que nos permite estudiar la satisfacibilidad de un problema. 

## Qué es CNF y porqué necesitamos entenderlo

PySAT solo recibe cosas en forma normal conjuntiva, es decir, expresiones de la forma:
$$(a \lor b) \land (c \lor d) \land ...$$

Donde cada parentesis se define como una $\textbf{clausula}$, la disyunción de $\textbf{literales}$, estas se unen entre si por medio de una conjunción.

Recuerden esta terminologia.

## Instalar PySAT

Para instalarlo, sigiendo las instrucciones disponibles en el [README](https://github.com/pysathq/pysat), primero debemos ejecutar los siguientes comandos:

In [ ]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 8.8 MB/s 
     |████████████████████████████████| 3.4 MB 56.5 MB/s 


In [ ]:
!pip install python-sat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U python-sat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Cómo entregar el input

Recordemos que PySAT solo recibe CNF, entonces debemos ingresar el input siguiendo esa logica.

Por ejemplo, tengo los literales $\{x_1, x_2, x_3, x_4\}$ y quiero estudiar la satisfacibilidad de:
$$(x_1 \lor x_2) \land (x_3 \lor x_4 \lor \neg x_1) \land \neg x_2$$

Para PySAT, $x_i$ se representa con $i$ y $\neg x_i$ con $-i$, de esta forma, definir la formula completa decimos:

In [ ]:
# Importamos el solver a usar
from pysat.solvers import Glucose3


In [ ]:
# Instanciamos el solver
s = Glucose3()

In [ ]:
# Añadiendo clausulas sin significado
s.add_clause([1, 2])

s.add_clause([3, -4])

In [ ]:
diccionario = {1: (1, 1, 'n'), 2: (1, 2, 'n'), 3: (1, 3, 'n'),
               4: (2, 1, 'n'), 5: (2, 2, 'n'), 6: (2, 3, 'n'),
               7: (1, 1, 'b'), 8: (1, 2, 'b'), 9: (1, 3, 'b'),
               10: (2, 1, 'b'), 11: (2, 2, 'b'), 12: (2, 3, 'b')}
print(diccionario)

{1: (1, 1, 'n'), 2: (1, 2, 'n'), 3: (1, 3, 'n'), 4: (2, 1, 'n'), 5: (2, 2, 'n'), 6: (2, 3, 'n'), 7: (3, 1, 'n'), 8: (3, 2, 'n'), 9: (3, 3, 'n')}


In [ ]:
# Finalmente se resuelve el problema
esSatisfacible = s.solve()

# 
valuacionQueSirve = s.get_model()
print(valuacionQueSirve)

[1, -2, -3, -4]


In [ ]:
from pysat.solvers import Glucose3


# Funcion auxiliar para manejar diccionarios
def findKeyByValue(dicc, value):
    return list(dicc.keys())[list(dicc.values()).index(value)]


def crearDiccionarioYPrimeraRestriccion(NP, s):
    diccionario = {}
    id = 0
    for i in range(len(NP)):  # Los subproblemas
        p_i = i + 1
        clausula = []
        for nodo in NP[i]:  # Los nodos
            id += 1
            diccionario[id] = (p_i, nodo, "n")  # Subproblema, nodo, es sobre nodos
            clausula.append(id)  # El indice de cada elemento representa el nodo en el que se debe
        # print(clausula)
        s.add_clause(clausula)  # Clausula 1

    return diccionario


def crearSegundaRestriccion(diccionario, NP, s):
    for i in range(len(NP)):
        p_i = i + 1
        for nodo_gorro in NP[i]:
            key_j = findKeyByValue(diccionario, (p_i, nodo_gorro, "n"))
            for key in diccionario:
                if nodo_gorro != diccionario[key][1] and p_i == diccionario[key][0] and diccionario[key][2] == "n":  # n_i != n_igorro
                    clausula2 = [-key_j, -key]
                    # print(clausula2)
                    s.add_clause(clausula2)


"""
AQUI EMPIEZA LA PARTE DE LOS BLOQUES
"""


def crearTerceraRestriccion(diccionario, P, B, s):
    id = len(diccionario)
    for p in P:
        clausula3 = []
        for bloque in B:
            id += 1
            clausula3.append(id)
            diccionario[id] = (p, bloque, "b")  # p,b
        # print(clausula3)
        s.add_clause(clausula3)


def crearCuartaRestriccion(diccionario, P, B, s):
    for p in P:
        for bloque in B:
            key_j = findKeyByValue(diccionario, (p, bloque, "b"))
            for key in diccionario:
                if bloque != diccionario[key][1] and p == diccionario[key][0] and "b" == diccionario[key][2]:
                    clausula4 = [-key_j, -key]
                    s.add_clause(clausula4)


def crearQuintaRestriccion(diccionario, P, B, Rp, R, s):
    Pr = calcularPr(Rp, R)
    for p in P:
        for b in B:
            key_j = findKeyByValue(diccionario, (p, b, "b"))
            recursosDeP = Rp[p - 1]
            for recurso in recursosDeP:
                subproblemasDelRecurso = Pr[recurso - 1]
                for pgorro in subproblemasDelRecurso:
                    if p != pgorro:
                        key = findKeyByValue(diccionario, (pgorro, b, "b"))
                        clausula5 = [-key_j, -key]
                        # print(clausula5)
                        s.add_clause(clausula5)


def crearSextaRestriccion(diccionario, P, NP, B, s):
    for p in P:
        N_p = NP[p - 1]
        for nodo in N_p:
            for b in B:
                key_beta = findKeyByValue(diccionario, (p, b, "b"))
                key_alpha = findKeyByValue(diccionario, (p, nodo, "n"))
                for pgorro in P:
                    if p != pgorro and (nodo in NP[pgorro - 1]):
                        key_betagorro = findKeyByValue(diccionario, (pgorro, b, "b"))
                        key_alphagorro = findKeyByValue(diccionario, (pgorro, nodo, "n"))
                        s.add_clause([-key_beta, -key_betagorro, -key_alpha, -key_alpha])


def calcularPr(Rp, R):
    P_r = []
    for r in R:
        fila = []
        for i in range(len(Rp)):
            if r in Rp[i]:
                fila.append(i + 1)
        P_r.append(fila)
    return P_r

print(calcularPr([[1,2,3]], [1, 2, 3]))

def generarModelo(P, NP, RP, R, tinit, tfinal, ttotal, t):
    bloques = []
    s = Glucose3()
    cantidadDeBloques = (ttotal - tinit - tfinal) // t
    for i in range(cantidadDeBloques):
        bloques.append(i + 1)
    diccionarioGeneral = crearDiccionarioYPrimeraRestriccion(NP, s)
    crearSegundaRestriccion(diccionarioGeneral, NP, s)
    crearTerceraRestriccion(diccionarioGeneral, P, bloques, s)
    crearCuartaRestriccion(diccionarioGeneral, P, bloques, s)
    crearQuintaRestriccion(diccionarioGeneral, P, bloques, RP, R, s)
    crearSextaRestriccion(diccionarioGeneral, P, NP, bloques, s)

    print("diccionario para los nodos:", diccionarioGeneral)

    esSatisfacible = s.solve()

    print("La formula definida es satisfacible? {}".format(esSatisfacible))

    valuacionQueSirve = s.get_model()
    if esSatisfacible:
        print("La asignacion de subproblemas debe ser la siguiente: ")
        for valuacion in valuacionQueSirve:
            if valuacion > 0:
                tuplaPB = diccionarioGeneral[valuacion]

                if tuplaPB[2] == "b":
                    print("el subproblema " + str(tuplaPB[0]) + " debe realizarse en el bloque " + str(tuplaPB[1]))
                elif tuplaPB[2] == "n":
                    print("el subproblema " + str(tuplaPB[0]) + " debe realizarse en el nodo " + str(tuplaPB[1]))
    s.delete()

"""
    PARTE C, INSTANCIAS SATISFACTIBLES Y NO SATISFACTIBLES
"""

# SET 1
P1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

N_p1 = [[1, 2, 3],  # Subproblema 1 -> 1, 2, 3
        [1, 2, 3],  # Subproblema 2 -> Puede ser resuelto en el nodo 1 2 o 3
        [1, 2, 3],
        [4, 5, 6],
        [4, 5, 6],
        [4, 5, 6],
        [7],  # Subproblema 7
        [7],  # Subproblema 8
        [9],
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

R1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

R_p1 = [[1, 2],  # Recursos de subproblema 1 R_p1
        [3, 4],
        [5, 6],
        [7, 8],
        [9, 10],
        [1, 2],
        [4],
        [5],
        [10],
        [10]]
t_total1 = 1200
t1 = 100
t_init1 = 100
t_final1 = 100

# SET 2
N_p2 = [[1],  # Solo tenemos 1 maquina y esta resuelve todos los subproblemas
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]]
N2 = [1]
generarModelo(P1, N_p1, R_p1, R1, t_init1, t_final1, t_total1, t1)
# generarModelo(P1, N_p2, R_p1, R1, t_init1, t_final1, t_total1, t1)

"""
    INSTANCIAS INSATISFACTIBLES 
"""
# No se pueden resolver todos los subproblemas
N_p3 = [[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [4, 5, 6],
        [4, 5, 6],
        [4, 5, 6],
        [7],
        [],  # Subproblema 8 no tiene un nodo que lo pueda resolver
        [9],
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

# Solo tenemos 1 bloque de tiempo para resolver 3 subproblemas => Insatisfactible
P4 = [1, 2, 3]
N4 = [1]
N_p4 = [[1],
        [1],
        [1]]
R4 = [1, 2, 3]
R_p4 = [[1],
        [2],
        [3]]
t_total4 = 300

# generarModelo(P1, N_p3, R_p1, R1, t_init1, t_final1, t_total1, t1)
# generarModelo(P3, N_p4, R_p4, R4, t_init1, t_final1, t_total4, t1)

[[1], [1], [1]]
diccionario para los nodos: {1: (1, 1, 'n'), 2: (1, 2, 'n'), 3: (1, 3, 'n'), 4: (2, 1, 'n'), 5: (2, 2, 'n'), 6: (2, 3, 'n'), 7: (3, 1, 'n'), 8: (3, 2, 'n'), 9: (3, 3, 'n'), 10: (4, 4, 'n'), 11: (4, 5, 'n'), 12: (4, 6, 'n'), 13: (5, 4, 'n'), 14: (5, 5, 'n'), 15: (5, 6, 'n'), 16: (6, 4, 'n'), 17: (6, 5, 'n'), 18: (6, 6, 'n'), 19: (7, 7, 'n'), 20: (8, 7, 'n'), 21: (9, 9, 'n'), 22: (10, 1, 'n'), 23: (10, 2, 'n'), 24: (10, 3, 'n'), 25: (10, 4, 'n'), 26: (10, 5, 'n'), 27: (10, 6, 'n'), 28: (10, 7, 'n'), 29: (10, 8, 'n'), 30: (10, 9, 'n'), 31: (10, 10, 'n'), 32: (1, 1, 'b'), 33: (1, 2, 'b'), 34: (1, 3, 'b'), 35: (1, 4, 'b'), 36: (1, 5, 'b'), 37: (1, 6, 'b'), 38: (1, 7, 'b'), 39: (1, 8, 'b'), 40: (1, 9, 'b'), 41: (1, 10, 'b'), 42: (2, 1, 'b'), 43: (2, 2, 'b'), 44: (2, 3, 'b'), 45: (2, 4, 'b'), 46: (2, 5, 'b'), 47: (2, 6, 'b'), 48: (2, 7, 'b'), 49: (2, 8, 'b'), 50: (2, 9, 'b'), 51: (2, 10, 'b'), 52: (3, 1, 'b'), 53: (3, 2, 'b'), 54: (3, 3, 'b'), 55: (3, 4, 'b'), 56: (3, 5, 'b'),

La formula definida es satisfacible? True
Qué valores de verdad deben tener cada uno de los literales para satisfacerlo? [1, -2, -3, -4, -5, -6]


Notar que nuestro solver (s) por debajo junta las clausulas con un and, la celda anterior es equivalente a la formula que escribimos más arriba.

Ahora, si quiero saber si la formula es satisfacible, le preguntamos al solver:


Si reemplazamos segun indica el solver, tenemos: $$\{x_1 = T, x_2 = F, x_3 = T, x_4 = F\}$$
Entonces:
$$(T \lor F) \land (T \lor F \lor \neg T) \land \neg F$$
$$(T \lor F) \land (T \lor F \lor F) \land T$$
$$T \land T \land T$$
$$T$$

Notar que esta valuación obtenida no necesariamente es unica, yo ahora ejecute la consulta y obtuve estos valores, pero quiza si vuelven a ejecutarlo tiene otra respuesta igual de válida :D

## Qué pasa si tengo una porrada de variables que no necesariamente son de la forma $x_i$

No pasa nada, si tienen cosas de la forma $x_{i,j,k,l}$ (por ejemplo) decidan la forma de numerarlas con un id, lo importante es que ustedes tengan muy claro como volver del id al literal original.

# (!) Importante
Pueden hacer una copia de este colab en sus drives y usarlo para correr sus códigos. A veces las instalaciones en sus computadores puede fallar porque tienen muchas librerias metidas y colisionan entre si.
Como dato, es buena practica hacer instalaciones higienicas para sus proyectos en python, para ello existe por ejemplo [venv](https://help.dreamhost.com/hc/es/articles/115000695551-Instalar-y-usar-virtualenv-con-Python-3), algo que podemos conversar otro día.